In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from skimage.measure import compare_ssim as ssim
import matplotlib.pyplot as plt
import time
import os
import copy
import h5py
import sys
import torch
from PIL import Image
import gc

sys.path.append('../../../code')

from python.finetune_commons import show_images, ft_train_model, visualize_model
from python.commons import load_dict_from_hdf5, save_dict_to_hdf5, inc_inference_e2e, full_inference_e2e, adaptive_drilldown, generate_heatmap
from python.vgg16 import VGG16

%matplotlib inline

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [25]:
n_labels = 2
gpu=True
patch_size = 32
beta = 0.7
image_size = 224
c = 0.0
file_path = '../../../data/oct/DRUSEN_NORMAL/validation/DRUSEN/DRUSEN-5333808-1.jpeg'

In [26]:
temp_weights_data = load_dict_from_hdf5('../../../exps/oct_drusen_vgg16_ptch.h5', gpu=True)
weights_data = load_dict_from_hdf5('../../../code/python/vgg16_weights_ptch.h5', gpu=True)
weights_data['fc8_W:0'] = temp_weights_data['fc8_W:0']
weights_data['fc8_b:0'] = temp_weights_data['fc8_b:0']

In [27]:
loader = transforms.Compose([transforms.Resize([image_size, image_size]), transforms.ToTensor()])
orig_image = Image.open(file_path).convert('RGB')
orig_image = loader(orig_image).unsqueeze(0)

if gpu:
    orig_image = orig_image.cuda()


image_patches = torch.FloatTensor(3, patch_size, patch_size).fill_(c).repeat(1, 1, 1, 1)

locations = torch.zeros([1, 2], dtype=torch.int32)
locations[0][0] = (image_size-patch_size)//2
locations[0][1] = (image_size-patch_size)//2

beta = 1.0
inc_model = VGG16(beta=beta, gpu=gpu, n_labels=n_labels, weights_data=weights_data)
inc_model.forward_materialized(orig_image)
logits_10 = inc_model.forward_gpu(image_patches, locations, p_height=patch_size, p_width=patch_size)

for beta in [1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]:
    
    orig_image = Image.open(file_path).convert('RGB')
    orig_image = loader(orig_image).unsqueeze(0)

    if gpu:
        orig_image = orig_image.cuda()


    image_patches = torch.FloatTensor(3, patch_size, patch_size).fill_(c).repeat(1, 1, 1, 1)
    
    locations = torch.zeros([1, 2], dtype=torch.int32)
    locations[0][0] = (image_size-patch_size)//2
    locations[0][1] = (image_size-patch_size)//2
    
    inc_model = VGG16(beta=beta, gpu=gpu, n_labels=n_labels, weights_data=weights_data)
    inc_model.forward_materialized(orig_image)

    logits = inc_model.forward_gpu(image_patches, locations, p_height=patch_size, p_width=patch_size)
        
    temp = logits_10 - logits
    temp = temp.cpu().data.numpy()
    temp = np.square(temp)
    temp = np.mean(temp)
    
    print(temp)

0.0
0.0
4.851637e-08
8.024915e-06
0.0021176692
0.0024987706
0.0025034887
0.0061079026
0.02723222
